In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from scipy.stats import ortho_group

In [2]:
data = pd.read_csv("/Users/lucastucker/REU-2023/archive/mnist_train.csv")
data = np.array(data)
np.random.shuffle(data)

In [3]:
data.shape

(60000, 785)

In [4]:
m = 2000 # number of MNIST images sampled
n = 784 # number of pixels per MNIST image
t = 50 # number of neighbors measured
sample = data[:m]
X = sample[:, 1:].T / 255
X.shape

(784, 2000)

In [5]:
def get_random_projection(k, X):
    R = np.random.normal(size = (k, n)) 
    frob_norm = np.linalg.norm(R.T.dot(R).dot(X) - X)
    print(f"frobenius norm for random pca is {frob_norm}")
    return R.dot(X)

In [6]:
def get_pca(k, X):
    mean_centered_data = X - np.mean(X, axis=1, keepdims=True)
    covariance_matrix = np.cov(mean_centered_data) 
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    proj = sorted_eigenvectors[:, :k] # top k minimize the frobenius norm
    frob_norm = np.linalg.norm(proj.dot(proj.T).dot(X) - X)
    print(f"frobenius norm for normal pca is {frob_norm}")
    reduced_data = np.dot(proj.T, mean_centered_data)
    return reduced_data

In [76]:
# This is performed according to the gradient calculated in the paper
def gradient_pca(k, X, tolerance, step):

    X = X - np.mean(X, axis=1, keepdims=True)
    diff = math.inf
    proj = ortho_group.rvs(dim=n)[:, :k] # random n x k orthogonal matrix
    norm = np.linalg.norm(proj)
    proj = proj / norm
    frob_norm = math.inf
    
    num_iterations = 20

    while diff > tolerance and num_iterations > 0:
        DF = 2 * proj.dot(proj.T).dot(X).dot(X.T).dot(proj) - \
        4 * X.dot(X.T).dot(proj) + 2 * X.dot(X.T).dot(proj).dot(proj.T).dot(proj)

        DF = DF / np.linalg.norm(DF) # Keep the gradient matrix normalized

        proj_new = (proj - step * DF) # gradient descent step
        diff = np.linalg.norm(proj - proj_new)
        proj = proj_new
        num_iterations -= 1
        # print(f"diff is {diff}") # how far we have moved 
        # frob_norm = np.linalg.norm(proj.dot(proj.T).dot(X) - X)
        # print(f"current frobenius is {frob_norm}") # current frobenius norm

    frob_norm = np.linalg.norm(proj.dot(proj.T).dot(X) - X)
    print(f"frobenius norm is {frob_norm}") # the objective function
    reduced_data = np.dot(proj.T, X)
    return reduced_data

In [8]:
def nearest_t_nbrs(t, X):
    t_nearest = np.ones((m, t), dtype=int) * 1
    for id, row in enumerate(X.T):
        dif = X.T - row # get vector representation-wise differences
        norm_indices = np.argsort(np.linalg.norm(dif, axis = 1))
        t_nearest[id] = norm_indices[1: t + 1]
    return t_nearest # returns m x t matrix representing k_nearest

In [9]:
def t_similarity_score(t, X, X_reduced):
    pixelwise_t_nearest = nearest_t_nbrs(t, X)
    reduced_t_nearest = nearest_t_nbrs(t, X_reduced)
    shared_elems_list = []
    for row, row_tilde in zip(pixelwise_t_nearest, reduced_t_nearest):
        set_1 = set(row)
        set_2 = set(row_tilde)
        shared_elem_count = len(set_1.intersection(set_2))
        shared_elems_list.append(shared_elem_count)
    shared_elems = np.array(shared_elems_list) # row-wise intersection counts
    avg_shared = (1/m) * np.sum(shared_elems)
    return (avg_shared / t)

In [10]:
k = 50

In [11]:
X_pca = get_pca(k, X)

frobenius norm for normal pca is 134.98551802433718


In [12]:
X_random = get_random_projection(k, X)

frobenius norm for random pca is 88368.25588007383


In [13]:
t_similarity_score(t, X, X_random)

0.58653

In [77]:
# got to under 133.5 with tol=0.0001, step=0.01
tol = 0.04
step = 1
X_gradient_pca = gradient_pca(k, X, tol, step)

frobenius norm is 188.60912789537159


In [78]:
t_similarity_score(t, X, X_gradient_pca)

0.7765000000000001